## Amazon Polly

In [ ]:
import boto3
import uuid
import subprocess
import json
import requests
import pandas

from contextlib import closing
from IPython.display import display, Markdown, HTML

In [ ]:
region = "<region>"

In [ ]:
def rand():
    return str(uuid.uuid4())[:8]

In [ ]:
def get(url):
    r = requests.get(url)
    return r.json()

### Conversión de texto a audio con Amazon Polly - polly.synthesize_speech( )

http://boto3.readthedocs.io/en/latest/reference/services/polly.html#Polly.Client.synthesize_speech

**Available voices** https://docs.aws.amazon.com/polly/latest/dg/voicelist.html

In [ ]:
def synthesize_speech(text, voice):
    polly = boto3.client('polly', region)
    response = polly.synthesize_speech(    
        OutputFormat='mp3',
        Text=text,
        TextType='text',
        VoiceId=voice
    )
    
    if "AudioStream" in response:
        with closing(response["AudioStream"]) as stream:
            output = "polly-{}.mp3".format(rand())

            try:
                with open(output, "wb") as file:
                    file.write(stream.read())
            except IOError as error:            
                print(error)
                sys.exit(-1)
    
    display(text)
    display(HTML("<audio controls><source src={} type='audio/mpeg'></audio>".format(output)))
            

synthesize_speech("<text>", "<voice>")


## Amazon Transcribe

### Conversión de audio a texto con Amazon Transcribe - transcribe.start_transcription_job( )

http://boto3.readthedocs.io/en/latest/reference/services/transcribe.html#TranscribeService.Client.start_transcription_job

In [ ]:
transcribe = boto3.client('transcribe',region)

def start_transcription_job(uri, settings={}):    
    
    job_name = 'Transcribe-Lab-{}'.format(rand())
    response = transcribe.start_transcription_job(
        TranscriptionJobName=job_name,
        LanguageCode='es-US',    
        MediaFormat='mp3',
        Media={
            'MediaFileUri': uri
        },
        Settings=settings
    )
    
    return job_name, response['TranscriptionJob']

job_name, response = start_transcription_job("https://s3-<region>.amazonaws.com/<bucket>/<key>")
display(response)

In [ ]:
response = transcribe.get_transcription_job(
    TranscriptionJobName=job_name
)

response['TranscriptionJob']

In [ ]:
get(response['TranscriptionJob']['Transcript']['TranscriptFileUri'])

## Amazon Comprehend

In [ ]:
comprehend = boto3.client('comprehend',region)

### Detección del idioma que predomina en el texto con Amazon Comprehend - detect_dominant_language( )

http://boto3.readthedocs.io/en/latest/reference/services/comprehend.html#Comprehend.Client.detect_dominant_language

In [ ]:
def detect_dominant_language(text):
    response = comprehend.detect_dominant_language(
        Text=text
    )
    return response['Languages'][0]['LanguageCode']

In [ ]:
LANG = detect_dominant_language("<text>")

display(Markdown("**El idioma detectado es: **{}".format(LANG)))

### Detección de las principales frases o palabras en el texto con Amazon Comprehend - detect_key_phrases( )

http://boto3.readthedocs.io/en/latest/reference/services/comprehend.html#Comprehend.Client.detect_key_phrases

In [ ]:
def detect_key_phrases(text, lang=None):
    if lang == None:
        lang = detect_dominant_language(text)
        
    response = comprehend.detect_key_phrases(
        Text=text,
        LanguageCode=lang
    )
    
    return response['KeyPhrases']

In [ ]:
response = detect_key_phrases("<text>")

display(pandas.DataFrame.from_dict(response))

### Detección de entidades en el texto con Amazon Comprehend - detect_entities( )

http://boto3.readthedocs.io/en/latest/reference/services/comprehend.html#Comprehend.Client.detect_entities

In [ ]:
def detect_entities(text, lang=None):
    if lang == None:
        lang = detect_dominant_language(text)
        
    response = comprehend.detect_entities(
        Text=text,
        LanguageCode=lang
    )
    
    return response['Entities']

In [ ]:
response = detect_entities("<text>")

display(pandas.DataFrame.from_dict(response))

### Detección del sentimiento en el texto con Amazon Comprehend - detect_sentiment( )

http://boto3.readthedocs.io/en/latest/reference/services/comprehend.html#Comprehend.Client.detect_sentiment

In [ ]:
def detect_sentiment(text, lang=None):
    if lang == None:
        lang = detect_dominant_language(text)
        
    response = comprehend.detect_sentiment(
        Text=text,
        LanguageCode=lang
    )
    
    return response['Sentiment'], response['SentimentScore']

In [ ]:
sentiment, scores = detect_sentiment("<text>")

display(Markdown("**El sentimiento detectado es:** {}".format(sentiment)))
display(Markdown("**El score de todos los sentimientos es:**"))
for score in scores:
    display(Markdown("&nbsp;&nbsp;&nbsp;**{0}:** {1}".format(score, scores[score]))) 

## Amazon Translate

In [ ]:
translate = boto3.client('translate')

### Traducción de texto con Amazon Translate - translate_text( ) 

http://boto3.readthedocs.io/en/latest/reference/services/translate.html#Translate.Client.translate_text

In [ ]:
def translate_text(text,source,target):
    response = translate.translate_text(
        Text=text,
        SourceLanguageCode=source,
        TargetLanguageCode=target
    )
    return response['TranslatedText']

In [ ]:
display(translate_text("<text>","es","en"))